In [24]:
%pip install backports.tarfile
%pip install zss

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Procesamiento de los datasets

In [25]:
import os
from collections import OrderedDict

base_path = 'IR-Plag-Dataset'
pairs_dict = OrderedDict()

for case in sorted(os.listdir(base_path)):
    case_path = os.path.join(base_path, case)
    if not os.path.isdir(case_path):
        continue

    dataset_name = 'IR-Plag'
    orig_path = os.path.join(case_path, 'original')
    orig_file = next(os.scandir(orig_path)).path
    orig_id   = f'{case}-ORIG'

    nonplag_path = os.path.join(case_path, 'non-plagiarized')
    if os.path.exists(nonplag_path):
        for folder in sorted(os.listdir(nonplag_path)):
            folder_path = os.path.join(nonplag_path, folder)
            if not os.path.isdir(folder_path):
                continue
            file_path = next(os.scandir(folder_path)).path
            file_id   = f'{case}-NP-{folder}'
            pair_id   = f'{orig_id}_{file_id}'
            pairs_dict[pair_id] = {
                'idcode1': orig_id,
                'code1'  : orig_file,
                'idcode2': file_id,
                'code2'  : file_path,
                'result' : 0,
                'dataset': dataset_name
            }

    plag_path = os.path.join(case_path, 'plagiarized')
    if os.path.exists(plag_path):
        for level in sorted(os.listdir(plag_path)):
            level_path = os.path.join(plag_path, level)
            for folder in sorted(os.listdir(level_path)):
                folder_path = os.path.join(level_path, folder)
                if not os.path.isdir(folder_path):
                    continue
                file_path = next(os.scandir(folder_path)).path
                file_id   = f'{case}-{level}-{folder}'
                pair_id   = f'{orig_id}_{file_id}'
                pairs_dict[pair_id] = {
                    'idcode1': orig_id,
                    'code1'  : orig_file,
                    'idcode2': file_id,
                    'code2'  : file_path,
                    'result' : 1,
                    'dataset': dataset_name
                }

print(len(pairs_dict))


460


In [26]:
import os
import pandas as pd

conplag_base     = 'conplag_version_2/versions'
conplag_code_dir = os.path.join(conplag_base, 'version_2')
conplag_labels   = os.path.join(conplag_base, 'labels.csv')

df = pd.read_csv(conplag_labels)

for _, row in df.iterrows():
    sub1, sub2, verdict = row['sub1'], row['sub2'], int(row['verdict'])

    codeid1  = str(sub1)
    codeid2  = str(sub2)
    pair_id  = f'{codeid1}_{codeid2}'

    folder_path = os.path.join(conplag_code_dir, pair_id)
    file1_path  = os.path.join(folder_path, f'{codeid1}.java')
    file2_path  = os.path.join(folder_path, f'{codeid2}.java')

    if not (os.path.exists(file1_path) and os.path.exists(file2_path)):
        continue

    pairs_dict[pair_id] = {
        'idcode1': codeid1,
        'code1'  : file1_path,          
        'idcode2': codeid2,
        'code2'  : file2_path,          
        'result' : verdict,             
        'dataset': 'conplag_version_2'
    }

print(f'✅ Total de pares tras añadir CONPLAG: {len(pairs_dict)}')

✅ Total de pares tras añadir CONPLAG: 1371


In [27]:
import os, random

base_path   = 'fire14-source-code-training-dataset'
java_path   = os.path.join(base_path, 'java')
qrel_path   = os.path.join(base_path, 'SOCO14-java.qrel')
dataset_name = 'FIRE14'

all_files = [f for f in os.listdir(java_path) if f.endswith('.java')]
file_set  = set(all_files)

positive_pairs = set()
with open(qrel_path, encoding='utf-8') as fh:
    for line in fh:
        f1, f2 = line.split()[:2]
        if not f1.endswith('.java'): f1 += '.java'
        if not f2.endswith('.java'): f2 += '.java'
        if f1 in file_set and f2 in file_set:
            positive_pairs.add((f1, f2))

positive_ids = list({x for p in positive_pairs for x in p})
negatives = set()
while len(negatives) < len(positive_pairs):
    f1, f2 = random.sample(positive_ids, 2)
    if f1 != f2 and (f1, f2) not in positive_pairs and (f2, f1) not in positive_pairs:
        negatives.add((f1, f2))

def add_pair(f1, f2, label):
    p1, p2 = os.path.join(java_path, f1), os.path.join(java_path, f2)
    if not (os.path.exists(p1) and os.path.exists(p2)): return
    pair_id = f'{f1}_{f2}'
    pairs_dict[pair_id] = {
        'idcode1': f1,
        'code1'  : p1,
        'idcode2': f2,
        'code2'  : p2,
        'result' : label,
        'dataset': dataset_name
    }

for f1, f2 in positive_pairs: add_pair(f1, f2, 1)
for f1, f2 in negatives:      add_pair(f1, f2, 0)

print(f'✅ Total de pares tras añadir FIRE14: {len(pairs_dict)}')


✅ Total de pares tras añadir FIRE14: 1539


# Calculo de features

In [28]:
def read_code(file_path):
    with open(file_path, encoding='utf-8') as fh:
        return fh.read()

In [29]:
import re, javalang
from simhash import Simhash
from collections import Counter

_token_pat = re.compile(r'[A-Za-z_]\w+|\d+|==|!=|<=|>=|&&|\|\||[^\sA-Za-z0-9_]')

def clean_code(src:str)->str:
    """Quita comentarios y espacios redundantes."""
    src = re.sub(r'//.*?$|/\*.*?\*/', '', src, flags=re.S|re.M)  # comentarios
    return '\n'.join(line.strip() for line in src.splitlines() if line.strip())

def text_features(src:str):
    """Devuelve (feature, weight) según frecuencia de cada token."""
    tokens = _token_pat.findall(clean_code(src))
    freq   = Counter(tokens)
    for tok, w in freq.items():
        yield tok, w                    # Simhash ya interpreta peso

def simhash_text(src:str)->Simhash:
    return Simhash(list(text_features(src)))

def text_similarity(code_a:str, code_b:str)->float:
    h1, h2 = simhash_text(code_a), simhash_text(code_b)
    return 1 - h1.distance(h2) / 64.0

def walk_ast(node):
    """Genera features estructurales: tipo de nodo y arista padre→hijo."""
    yield node.__class__.__name__
    for child in node.children:
        if isinstance(child, list):
            for g in child:
                if isinstance(g, javalang.ast.Node):
                    yield f'{node.__class__.__name__}->{g.__class__.__name__}'
                    yield from walk_ast(g)
        elif isinstance(child, javalang.ast.Node):
            yield f'{node.__class__.__name__}->{child.__class__.__name__}'
            yield from walk_ast(child)

def simhash_ast(src:str)->Simhash:
    ast   = javalang.parse.parse(src)
    feats = list(walk_ast(ast))
    return Simhash(feats)

def ast_similarity(code_a:str, code_b:str)->float:
    h1, h2 = simhash_ast(code_a), simhash_ast(code_b)
    return 1 - h1.distance(h2) / 64.0


In [30]:
from tqdm import tqdm 

rows = []
total = len(pairs_dict)

for pid, info in tqdm(pairs_dict.items(),
                      total=total,
                      desc="Procesando pares"):
    try:
        # Usa tu propia función de lectura
        code_a = read_code(info["code1"])
        code_b = read_code(info["code2"])

        # ­Features
        simhash_sim = text_similarity(code_a, code_b)
        sim_ast = ast_similarity(code_a, code_b)
        print(f"🔍 {pid} - Simhash: {simhash_sim:.4f}, Similitud ASTs: {sim_ast:.4f}")

        rows.append({
            "id"           : pid,
            "idcode1"      : info["idcode1"],
            "idcode2"      : info["idcode2"],
            "code1"        : info["code1"],   # ruta
            "code2"        : info["code2"],   # ruta
            "simhash"      : simhash_sim,
            "astsimilarity": sim_ast,
            "dataset"      : info["dataset"],
            "result"       : info["result"],
        })

    except Exception as e:
        # Cualquier fallo de parseo/similaridad ⇒ se omite el par
        print(f"⚠️  {pid} omitido ({e})")

# Guarda el CSV
import pandas as pd
pd.DataFrame(rows).to_csv("dataset.csv", index=False)
print("✅  dataset.csv generado con", len(rows), "filas")


Procesando pares:   4%|▍         | 60/1539 [00:00<00:02, 583.91it/s]

🔍 case-01-ORIG_case-01-NP-01 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-02 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-03 - Simhash: 0.7500, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-04 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-05 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-06 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-07 - Simhash: 0.7500, Similitud ASTs: 0.7188
🔍 case-01-ORIG_case-01-NP-08 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-09 - Simhash: 0.7656, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-10 - Simhash: 0.7656, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-11 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-12 - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 case-01-ORIG_case-01-NP-13 - Simhash: 0.9062, Similitud ASTs: 1.0000
🔍 case-01-ORIG_case-01-NP-14 - Simhash: 0.7344, Similitud ASTs: 0.7188
🔍 case

Procesando pares:   8%|▊         | 119/1539 [00:00<00:03, 463.53it/s]

🔍 case-02-ORIG_case-02-L5-07 - Simhash: 0.8125, Similitud ASTs: 0.7969
🔍 case-02-ORIG_case-02-L5-08 - Simhash: 0.9375, Similitud ASTs: 0.8750
🔍 case-02-ORIG_case-02-L5-09 - Simhash: 0.9062, Similitud ASTs: 0.8594
🔍 case-02-ORIG_case-02-L6-01 - Simhash: 0.8750, Similitud ASTs: 0.7656
🔍 case-02-ORIG_case-02-L6-02 - Simhash: 0.8594, Similitud ASTs: 0.7344
🔍 case-02-ORIG_case-02-L6-03 - Simhash: 0.8750, Similitud ASTs: 0.7656
🔍 case-02-ORIG_case-02-L6-04 - Simhash: 0.9375, Similitud ASTs: 0.8750
🔍 case-02-ORIG_case-02-L6-05 - Simhash: 0.8594, Similitud ASTs: 0.7500
🔍 case-02-ORIG_case-02-L6-06 - Simhash: 0.8438, Similitud ASTs: 0.8125
🔍 case-02-ORIG_case-02-L6-07 - Simhash: 0.8438, Similitud ASTs: 0.8125
🔍 case-02-ORIG_case-02-L6-08 - Simhash: 0.9219, Similitud ASTs: 0.8750
🔍 case-02-ORIG_case-02-L6-09 - Simhash: 0.8438, Similitud ASTs: 0.8281
🔍 case-03-ORIG_case-03-NP-01 - Simhash: 0.9219, Similitud ASTs: 0.9531
🔍 case-03-ORIG_case-03-NP-02 - Simhash: 0.9375, Similitud ASTs: 0.9531
🔍 case

Procesando pares:  11%|█         | 167/1539 [00:00<00:04, 342.60it/s]

...

🔍 032.java_087.java - Simhash: 0.6719, Similitud ASTs: 0.7812
🔍 137.java_182.java - Simhash: 0.8281, Similitud ASTs: 0.8750
🔍 050.java_232.java - Simhash: 0.7031, Similitud ASTs: 0.7500
🔍 140.java_258.java - Simhash: 0.7656, Similitud ASTs: 0.8594
🔍 185.java_103.java - Simhash: 0.7031, Similitud ASTs: 0.9062
🔍 033.java_190.java - Simhash: 0.6250, Similitud ASTs: 0.7031
🔍 022.java_079.java - Simhash: 0.6719, Similitud ASTs: 0.7500
🔍 111.java_107.java - Simhash: 0.7969, Similitud ASTs: 0.7969
🔍 188.java_183.java - Simhash: 0.6719, Similitud ASTs: 0.7344
🔍 021.java_107.java - Simhash: 0.8906, Similitud ASTs: 0.7188
🔍 148.java_084.java - Simhash: 0.8594, Similitud ASTs: 0.8281
🔍 190.java_143.java - Simhash: 0.7500, Similitud ASTs: 0.7969
🔍 150.java_089.java - Simhash: 0.9062, Similitud ASTs: 0.8750
🔍 117.java_201.java - Simhash: 0.7812, Similitud ASTs: 0.8125
🔍 113.java_086.java - Simhash: 0.6094, Similitud ASTs: 0.6250


Procesando pares:  99%|█████████▉| 1524/1539 [00:12<00:00, 78.77it/s]

🔍 070.java_242.java - Simhash: 0.7188, Similitud ASTs: 0.8281
🔍 024.java_032.java - Simhash: 0.6562, Similitud ASTs: 0.8125
🔍 188.java_240.java - Simhash: 0.7812, Similitud ASTs: 0.7969
🔍 146.java_089.java - Simhash: 0.8906, Similitud ASTs: 0.8750
🔍 079.java_209.java - Simhash: 0.8438, Similitud ASTs: 0.7812
🔍 094.java_230.java - Simhash: 0.8281, Similitud ASTs: 0.8750
🔍 133.java_014.java - Simhash: 0.7969, Similitud ASTs: 0.9375
🔍 086.java_078.java - Simhash: 0.5469, Similitud ASTs: 0.9062
🔍 042.java_136.java - Simhash: 0.8125, Similitud ASTs: 0.8594
🔍 211.java_078.java - Simhash: 0.9375, Similitud ASTs: 0.9531
🔍 059.java_015.java - Simhash: 0.6875, Similitud ASTs: 0.6562
🔍 047.java_212.java - Simhash: 0.7656, Similitud ASTs: 0.8750
🔍 218.java_159.java - Simhash: 0.5938, Similitud ASTs: 0.7344
🔍 216.java_051.java - Simhash: 0.6875, Similitud ASTs: 0.8281
🔍 195.java_094.java - Simhash: 0.7969, Similitud ASTs: 0.8906
🔍 111.java_250.java - Simhash: 0.5938, Similitud ASTs: 0.7031
🔍 004.ja

Procesando pares: 100%|██████████| 1539/1539 [00:12<00:00, 118.87it/s]

🔍 008.java_251.java - Simhash: 0.8438, Similitud ASTs: 0.8594
🔍 089.java_112.java - Simhash: 0.7969, Similitud ASTs: 0.7031
🔍 188.java_173.java - Simhash: 0.8125, Similitud ASTs: 0.7188
🔍 211.java_131.java - Simhash: 0.5156, Similitud ASTs: 0.9688
🔍 133.java_221.java - Simhash: 0.6250, Similitud ASTs: 0.8281
🔍 143.java_131.java - Simhash: 0.7031, Similitud ASTs: 0.7656
🔍 119.java_147.java - Simhash: 0.7812, Similitud ASTs: 0.9062
🔍 171.java_230.java - Simhash: 0.8438, Similitud ASTs: 0.9688
🔍 004.java_087.java - Simhash: 0.6562, Similitud ASTs: 0.7188
🔍 173.java_023.java - Simhash: 0.6562, Similitud ASTs: 0.6562
🔍 053.java_033.java - Simhash: 0.7188, Similitud ASTs: 0.7656
🔍 047.java_064.java - Simhash: 0.8438, Similitud ASTs: 0.9219
🔍 135.java_175.java - Simhash: 0.7188, Similitud ASTs: 0.7500
🔍 051.java_016.java - Simhash: 0.9375, Similitud ASTs: 0.9375
🔍 232.java_034.java - Simhash: 0.5469, Similitud ASTs: 0.7031
✅  dataset.csv generado con 1538 filas
